# cnt_month와 lag 모두 clip한 데이터

In [1]:
# 라이브러리 호출
import numpy as np
import pandas as pd
import time
from itertools import product 
import lightgbm

In [2]:
data = pd.read_pickle('./data/data_1.pkl')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8032887 entries, 827691 to 8860577
Data columns (total 29 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   month_id                    int8   
 1   shop_id                     int8   
 2   item_id                     int16  
 3   cnt_month                   float32
 4   city_id                     int8   
 5   cat_id                      int8   
 6   main_cat                    int8   
 7   sub_cat                     int8   
 8   year                        int16  
 9   month                       int8   
 10  sales_month                 int8   
 11  cnt_month_1                 float32
 12  cnt_month_2                 float32
 13  cnt_month_3                 float32
 14  item_id_mean_cnt_1          float32
 15  item_id_mean_cnt_2          float32
 16  item_id_mean_cnt_3          float32
 17  sub_cat_mean_cnt_1          float32
 18  sub_cat_mean_cnt_2          float32
 19  sub_cat_mean_cnt

In [3]:
data.isnull().sum()

month_id                           0
shop_id                            0
item_id                            0
cnt_month                     214200
city_id                            0
cat_id                             0
main_cat                           0
sub_cat                            0
year                               0
month                              0
sales_month                        0
cnt_month_1                        0
cnt_month_2                        0
cnt_month_3                        0
item_id_mean_cnt_1                 0
item_id_mean_cnt_2                 0
item_id_mean_cnt_3                 0
sub_cat_mean_cnt_1                 0
sub_cat_mean_cnt_2                 0
sub_cat_mean_cnt_3                 0
price_mean_mean_cnt_1              0
price_mean_mean_cnt_2              0
price_mean_mean_cnt_3              0
item_id/city_id_mean_cnt_1         0
item_id/city_id_mean_cnt_2         0
item_id/city_id_mean_cnt_3         0
shop_id/cat_id_mean_cnt_1          0
s

In [5]:
data[['cnt_month','cnt_month_1','cnt_month_2','cnt_month_3']]=data[['cnt_month','cnt_month_1','cnt_month_2','cnt_month_3']].clip(0,20)

In [6]:
# train/validation/test data split
X_train = data[data['month_id'] < 33].drop(['cnt_month'], axis=1)
y_train = data[data['month_id'] < 33]['cnt_month']

X_valid = data[data['month_id'] == 33].drop(['cnt_month'], axis=1)
y_valid = data[data['month_id'] == 33]['cnt_month']

X_test = data[data['month_id'] == 34].drop(['cnt_month'], axis=1)

In [7]:
# hyper parameter 지정
params = {'metric': 'rmse', # 평가지표 = rmse
          'num_leaves': 255,
          'learning_rate': 0.005,
          'feature_fraction': 0.75,
          'bagging_fraction': 0.75,
          'bagging_freq': 5,
          'force_col_wise': True,
          'random_state': 42}

cat = ['shop_id', 'city_id', 'cat_id', 'main_cat', 'sub_cat']

# 데이터 셋 지정
train = lightgbm.Dataset(X_train, y_train)
valid = lightgbm.Dataset(X_valid, y_valid)
 
# 모델 훈련
start = time.time()
lgb_model = lightgbm.train(params=params,
                      train_set=train,
                      num_boost_round=1500,
                      valid_sets=(train, valid),
                      early_stopping_rounds=150,
                      categorical_feature=cat,
                      verbose_eval=50)
end = time.time()
print(f'소요시간(s) : {end-start}')

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat_id', 'city_id', 'main_cat', 'shop_id', 'sub_cat']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Total Bins 4304
[LightGBM] [Info] Number of data points in the train set: 7596885, number of used features: 28


C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Info] Start training from score 0.306474
Training until validation scores don't improve for 150 rounds
[50]	training's rmse: 1.10765	valid_1's rmse: 0.980022
[100]	training's rmse: 1.02054	valid_1's rmse: 0.914853
[150]	training's rmse: 0.960453	valid_1's rmse: 0.875182
[200]	training's rmse: 0.918499	valid_1's rmse: 0.850936
[250]	training's rmse: 0.888935	valid_1's rmse: 0.836415
[300]	training's rmse: 0.867369	valid_1's rmse: 0.829276
[350]	training's rmse: 0.851587	valid_1's rmse: 0.824903
[400]	training's rmse: 0.839453	valid_1's rmse: 0.822748
[450]	training's rmse: 0.829519	valid_1's rmse: 0.822066
[500]	training's rmse: 0.821795	valid_1's rmse: 0.822034
[550]	training's rmse: 0.815173	valid_1's rmse: 0.822322
Early stopping, best iteration is:
[436]	training's rmse: 0.832159	valid_1's rmse: 0.821817
소요시간(s) : 359.2579290866852


In [11]:
# 예측 수행
y_pred = lgb_model.predict(X_test).clip(0, 20) # 실제 타겟값이 클립되어있으므로 동일하게 클립

In [12]:
# 제출 파일 생성
df_test = pd.read_csv('e:/Git_public_dodo_Riley/kaggle/Future_Sales/data/test.csv')
submission = pd.DataFrame({"ID": df_test.index, "item_cnt_month": y_pred})
submission.to_csv('lgbm_submission_2.csv', index=False)

In [ ]:
# Score: 0.89709, 2010/13595~상위 14.8%